In [1]:
import pandas as pd
dataRecorridos = pd.read_csv("../data/raw/badata_ecobici_recorridos_realizados_2024.csv")
dataUsuarios = pd.read_csv("../data/raw/usuarios_ecobici_2024.csv")

## Filtrado, preparación e integración de datos

Creamos una funcion para renombrar las columnas a camelCase

In [2]:
def to_camel_case(df):
  new_columns = {}
  for col in df.columns:
    parts = col.split('_')
    new_col = parts[0] + ''.join(word.capitalize() for word in parts[1:])
    new_columns[col] = new_col
  return df.rename(columns=new_columns)

dataRecorridos = to_camel_case(dataRecorridos)
dataUsuarios = to_camel_case(dataUsuarios)

print("Columnas de dataRecorridos en CamelCase:")
print(dataRecorridos.columns)
print("\nColumnas de dataUsuarios en CamelCase:")
print(dataUsuarios.columns)

Columnas de dataRecorridos en CamelCase:
Index(['idRecorrido', 'duracionRecorrido', 'fechaOrigenRecorrido',
       'idEstacionOrigen', 'nombreEstacionOrigen', 'direccionEstacionOrigen',
       'longEstacionOrigen', 'latEstacionOrigen', 'fechaDestinoRecorrido',
       'idEstacionDestino', 'nombreEstacionDestino',
       'direccionEstacionDestino', 'longEstacionDestino', 'latEstacionDestino',
       'idUsuario', 'modeloBicicleta', 'genero'],
      dtype='object')

Columnas de dataUsuarios en CamelCase:
Index(['idUsuario', 'generoUsuario', 'edadUsuario', 'fechaAlta', 'horaAlta'], dtype='object')


Convertimos las columnas de fechas a datetime

In [3]:
dataRecorridos['fechaOrigenRecorrido'] = pd.to_datetime(dataRecorridos['fechaOrigenRecorrido'])
dataRecorridos['fechaDestinoRecorrido'] = pd.to_datetime(dataRecorridos['fechaDestinoRecorrido'])

dataUsuarios['fechaAlta'] = pd.to_datetime(dataUsuarios['fechaAlta'])
dataUsuarios['horaAlta'] = pd.to_datetime(dataUsuarios['horaAlta']).dt.time

C:\Users\tomas\AppData\Local\Temp\ipykernel_3108\2643729637.py:5: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  dataUsuarios['horaAlta'] = pd.to_datetime(dataUsuarios['horaAlta']).dt.time


Filtramos y guardamos viajes de julio, agosto y septiembre de 2024

In [4]:
mask = dataRecorridos['fechaOrigenRecorrido'].dt.month.isin([7, 8, 9]) & (dataRecorridos['fechaOrigenRecorrido'].dt.year == 2024)
dataRecorridos_jul_sep = dataRecorridos[mask]
dataRecorridos_jul_sep.head()

,idRecorrido,duracionRecorrido,fechaOrigenRecorrido,idEstacionOrigen,nombreEstacionOrigen,direccionEstacionOrigen,longEstacionOrigen,latEstacionOrigen,fechaDestinoRecorrido,idEstacionDestino,nombreEstacionDestino,direccionEstacionDestino,longEstacionDestino,latEstacionDestino,idUsuario,modeloBicicleta,genero
27968,22751201,2,2024-08-24 12:37:54,548,051 - PLAZA SUDAMERICA,Av. Piedrabuena 4901,-58.475178,-34.687767,2024-08-24 12:37:56,548,051 - PLAZA SUDAMERICA,Av. Piedrabuena 4901,-58.475178,-34.687767,1195811.0,ICONIC,MALE
44853,22526638,1350,2024-08-02 01:29:21,46,046 - Chile,Perú 718,-58.374370,-34.616373,2024-08-02 01:51:51,223,178 - José Marmol,803 José Marmol,-58.424853,-34.623400,1126622.0,FIT,FEMALE
44854,22527526,1398,2024-08-02 07:26:40,408,364 - CERRETTI Y MANUELA PEDRAZA,Ceretti 3407,-58.500250,-34.569120,2024-08-02 07:49:58,513,308 - SAN MARTIN II,Av. San Martín 5129,-58.490739,-34.597130,569548.0,FIT,MALE
44855,22517654,491,2024-08-01 12:51:39,368,378 - AGRONOMIA,4351 San Martin,-58.482079,-34.598070,2024-08-01 12:59:50,241,348 - Villa del Parque,"Gutierrez, Ricardo 3105",-58.494123,-34.600874,569548.0,ICONIC,MALE
44856,22524071,1642,2024-08-01 19:12:28,385,268 - Plaza Aristóbulo Del Valle,Campana & Marcos Sastre,-58.493141,-34.606160,2024-08-01 19:39:50,408,364 - CERRETTI Y MANUELA PEDRAZA,Ceretti 3407,-58.500250,-34.569120,569548.0,FIT,MALE


Creamos un CSV con los datos filtrados

In [5]:
dataRecorridos_jul_sep.to_csv("../data/interim/recorridos_3meses.csv", index=False)

Verificamos si los valores de idUsuario coinciden en ambos DataFrames

In [6]:
print("\n¿'idUsuario' está en ambos DataFrames?")
print('idUsuario' in dataRecorridos.columns and 'idUsuario' in dataUsuarios.columns)


¿'idUsuario' está en ambos DataFrames?
True


Realizamos un join entre dataRecorridos_jul_sep y dataUsuarios usando 'idUsuario'

In [7]:
recorridos_usuarios = dataRecorridos_jul_sep.merge(
    dataUsuarios,
    on='idUsuario',
    how='left',
    suffixes=('', '_usuario')
)
recorridos_usuarios.head()

,idRecorrido,duracionRecorrido,fechaOrigenRecorrido,idEstacionOrigen,nombreEstacionOrigen,direccionEstacionOrigen,longEstacionOrigen,latEstacionOrigen,fechaDestinoRecorrido,idEstacionDestino,...,direccionEstacionDestino,longEstacionDestino,latEstacionDestino,idUsuario,modeloBicicleta,genero,generoUsuario,edadUsuario,fechaAlta,horaAlta
0,22751201,2,2024-08-24 12:37:54,548,051 - PLAZA SUDAMERICA,Av. Piedrabuena 4901,-58.475178,-34.687767,2024-08-24 12:37:56,548,...,Av. Piedrabuena 4901,-58.475178,-34.687767,1195811.0,ICONIC,MALE,MALE,44.0,2024-08-24,12:32:34
1,22526638,1350,2024-08-02 01:29:21,46,046 - Chile,Perú 718,-58.374370,-34.616373,2024-08-02 01:51:51,223,...,803 José Marmol,-58.424853,-34.623400,1126622.0,FIT,FEMALE,FEMALE,22.0,2024-03-11,00:53:54
2,22527526,1398,2024-08-02 07:26:40,408,364 - CERRETTI Y MANUELA PEDRAZA,Ceretti 3407,-58.500250,-34.569120,2024-08-02 07:49:58,513,...,Av. San Martín 5129,-58.490739,-34.597130,569548.0,FIT,MALE,NaN,NaN,NaT,NaN
3,22517654,491,2024-08-01 12:51:39,368,378 - AGRONOMIA,4351 San Martin,-58.482079,-34.598070,2024-08-01 12:59:50,241,...,"Gutierrez, Ricardo 3105",-58.494123,-34.600874,569548.0,ICONIC,MALE,NaN,NaN,NaT,NaN
4,22524071,1642,2024-08-01 19:12:28,385,268 - Plaza Aristóbulo Del Valle,Campana & Marcos Sastre,-58.493141,-34.606160,2024-08-01 19:39:50,408,...,Ceretti 3407,-58.500250,-34.569120,569548.0,FIT,MALE,NaN,NaN,NaT,NaN


Creamos la columna derivada duracion_viaje_minutos

In [ ]:
# Función para convertir minutos a formato "Xh Ym"
def minutos_a_horas(minutos):
    horas = minutos // 60  # División entera para obtener las horas
    resto_minutos = minutos % 60  # Resto para los minutos
    return f"{horas}h {resto_minutos}m"

In [13]:
recorridos_usuarios['duracion_viaje_minutos'] = recorridos_usuarios['duracionRecorrido'].apply(minutos_a_horas)
recorridos_usuarios.head()

,idRecorrido,duracionRecorrido,fechaOrigenRecorrido,idEstacionOrigen,nombreEstacionOrigen,direccionEstacionOrigen,longEstacionOrigen,latEstacionOrigen,fechaDestinoRecorrido,idEstacionDestino,...,latEstacionDestino,idUsuario,modeloBicicleta,genero,generoUsuario,edadUsuario,fechaAlta,horaAlta,duracion_horas_minutos,duracion_viaje_minutos
0,22751201,2,2024-08-24 12:37:54,548,051 - PLAZA SUDAMERICA,Av. Piedrabuena 4901,-58.475178,-34.687767,2024-08-24 12:37:56,548,...,-34.687767,1195811.0,ICONIC,MALE,MALE,44.0,2024-08-24,12:32:34,0h 2m,0h 2m
1,22526638,1350,2024-08-02 01:29:21,46,046 - Chile,Perú 718,-58.374370,-34.616373,2024-08-02 01:51:51,223,...,-34.623400,1126622.0,FIT,FEMALE,FEMALE,22.0,2024-03-11,00:53:54,22h 30m,22h 30m
2,22527526,1398,2024-08-02 07:26:40,408,364 - CERRETTI Y MANUELA PEDRAZA,Ceretti 3407,-58.500250,-34.569120,2024-08-02 07:49:58,513,...,-34.597130,569548.0,FIT,MALE,NaN,NaN,NaT,NaN,23h 18m,23h 18m
3,22517654,491,2024-08-01 12:51:39,368,378 - AGRONOMIA,4351 San Martin,-58.482079,-34.598070,2024-08-01 12:59:50,241,...,-34.600874,569548.0,ICONIC,MALE,NaN,NaN,NaT,NaN,8h 11m,8h 11m
4,22524071,1642,2024-08-01 19:12:28,385,268 - Plaza Aristóbulo Del Valle,Campana & Marcos Sastre,-58.493141,-34.606160,2024-08-01 19:39:50,408,...,-34.569120,569548.0,FIT,MALE,NaN,NaN,NaT,NaN,27h 22m,27h 22m


Creamos un CSV con los datos procesados

In [14]:
recorridos_usuarios.to_csv("../data/processed/recorridos_usuarios_3meses.csv", index=False)